In [1]:
import numpy as np
import pandas as pd

In [2]:
df1 = pd.read_csv("D:/ml/deepwalk-master/kospi200/data/kospi200.csv",
                 index_col = 0,
                 header=0,
                 encoding='cp949', sep=',')

In [3]:
df1.head()

,시가,고가,저가,현재가,거래량
일자,,,,,
2020-01-31,290.24,291.47,284.53,284.53,"101,455"
2020-01-30,293.27,294.11,287.09,288.37,"101,535"
2020-01-29,294.38,295.67,292.45,293.98,"85,731"
2020-01-28,294.98,296.30,291.30,292.77,"130,172"
2020-01-23,303.77,304.72,301.71,302.33,"86,908"


In [4]:
print(df1.shape)

(426, 5)


In [5]:
df2 = pd.read_csv("D:/ml/deepwalk-master/kospi200/data/samsung.csv",
                 index_col = 0,
                 header=0,
                 encoding='cp949', sep=',')

In [6]:
df2.head()

,시가,고가,저가,종가,거래량
일자,,,,,
2020-01-31,"57,800","58,400","56,400","56,400","19,749,457"
2020-01-30,"58,800","58,800","56,800","57,200","20,821,939"
2020-01-29,"59,100","59,700","58,800","59,100","16,446,102"
2020-01-28,"59,400","59,400","58,300","58,800","23,664,541"
2020-01-23,"61,800","61,800","60,700","60,800","14,916,555"


In [7]:
print(df2.shape)

(426, 5)


In [8]:
df1.dtypes

시가     float64
고가     float64
저가     float64
현재가    float64
거래량     object
dtype: object

In [9]:
df2.dtypes

시가     object
고가     object
저가     object
종가     object
거래량    object
dtype: object

In [10]:
for i in range(len(df1.index)):
    
    df1.iloc[i,4] = int(df1.iloc[i,4].replace(',',''))

In [11]:
for i in range(len(df2.index)):
    
    for j in range(len(df2.iloc[i])):
        
        df2.iloc[i,j] = int(df2.iloc[i,j].replace(',',''))

In [12]:
df1 = df1.sort_values(['일자'], ascending=True)
df2 = df2.sort_values(['일자'], ascending=True)

In [13]:
df1['거래량'] = df1['거래량'].astype('float64')

In [14]:
df2['시가'] = df2['시가'].astype('float64')
df2['고가'] = df2['고가'].astype('float64')
df2['저가'] = df2['저가'].astype('float64')
df2['종가'] = df2['종가'].astype('float64')
df2['거래량'] = df2['거래량'].astype('float64')

In [15]:
df1 = df1.values
df2 = df2.values

In [16]:
type(df1)

numpy.ndarray

In [17]:
type(df2)

numpy.ndarray

In [18]:
kospi200 = df1.copy()
samsung = df2.copy()

In [20]:
def split_xy5(dataset, time_steps, y_column):
    x, y = list(), list()
    for i in range(len(dataset)):
        x_end_number = i + time_steps
        y_end_number = x_end_number + y_column # 수정

        if y_end_number > len(dataset):  # 수정
            break
        tmp_x = dataset[i:x_end_number, :]  # 수정
        tmp_y = dataset[x_end_number:y_end_number, 3]    # 수정
        x.append(tmp_x)
        y.append(tmp_y)
    return np.array(x), np.array(y)


In [21]:
x, y = split_xy5(samsung, 5, 1) 

In [22]:
print(x[0,:], "\n", y[0])
print(x.shape)
print(y.shape)

[[   53000.    53900.    51800.    51900. 39565391.]
 [   52600.    53200.    51900.    52600. 23104720.]
 [   52600.    52800.    50900.    50900. 16128305.]
 [   51700.    51700.    50600.    51600. 13905263.]
 [   52000.    52200.    51200.    51300. 10314997.]] 
 [50100.]
(421, 5, 5)
(421, 1)


In [23]:
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score
x_train, x_test, y_train, y_test = train_test_split(
    x, y, random_state=1, test_size = 0.3)

In [24]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(294, 5, 5)
(127, 5, 5)
(294, 1)
(127, 1)


In [25]:
x_train = np.reshape(x_train,
                     (x_train.shape[0], x_train.shape[1] * x_train.shape[2]))
x_test = np.reshape(x_test,
                    (x_test.shape[0], x_test.shape[1] * x_test.shape[2]))

In [26]:
print(x_train.shape)
print(x_test.shape)

(294, 25)
(127, 25)


In [27]:
from sklearn.preprocessing import StandardScaler

In [28]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [29]:
print(x_train_scaled[0, :])

[ 1.13392563  1.07633086  1.02393741  0.93187013 -0.028819    1.05994459
  1.04756614  1.06686328  0.94933344  1.53138774  0.79093879  0.80213886
  0.84540479  0.81507608  0.10811605  0.73712199  0.68893234  0.71073997
  0.69657856  0.37360361  0.85647275  0.80021877  0.80910994  0.69442952
 -0.16973555]


In [31]:
x_train_scaled = np.reshape(x_train_scaled,
                            (x_train_scaled.shape[0], 5, 5))
x_test_scaled = np.reshape(x_test_scaled,
                           (x_test_scaled.shape[0], 5, 5))

In [32]:
print(x_train_scaled.shape)
print(x_test_scaled.shape)

(294, 5, 5)
(127, 5, 5)


In [33]:
from keras.models import Sequential
from keras.layers import Dense, LSTM


Using TensorFlow backend.


In [34]:
model = Sequential()

In [35]:
model.add(LSTM(64, input_shape=(5,5)))

In [36]:
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

In [37]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])

In [38]:
from tensorflow.keras.callbacks import EarlyStopping

In [39]:
early_stopping = EarlyStopping(patience =20)

In [40]:
model.fit(x_train_scaled, y_train, validation_split=0.2, verbose=1,
         batch_size = 1, epochs = 100, callbacks=[early_stopping])

Epoch 1/100
235/235 [==============================] - 1s 3ms/step - loss: 1624556032.0000 - mse: 1624556032.0000 - val_loss: 35238156.0000 - val_mse: 35238156.0000
Epoch 2/100
235/235 [==============================] - 0s 2ms/step - loss: 17221282.0000 - mse: 17221282.0000 - val_loss: 4831198.0000 - val_mse: 4831198.0000
Epoch 3/100
235/235 [==============================] - 0s 2ms/step - loss: 4219512.5000 - mse: 4219512.5000 - val_loss: 2938135.5000 - val_mse: 2938135.5000
Epoch 4/100
235/235 [==============================] - 0s 2ms/step - loss: 3020389.5000 - mse: 3020389.5000 - val_loss: 2533525.7500 - val_mse: 2533525.7500
Epoch 5/100
235/235 [==============================] - 0s 2ms/step - loss: 2690835.2500 - mse: 2690835.2500 - val_loss: 2178080.2500 - val_mse: 2178080.2500
Epoch 6/100
235/235 [==============================] - 0s 2ms/step - loss: 2417516.7500 - mse: 2417516.7500 - val_loss: 2002138.1250 - val_mse: 2002138.1250
Epoch 7/100
235/235 [===========================

235/235 [==============================] - 0s 2ms/step - loss: 803791.8750 - mse: 803791.8750 - val_loss: 652193.3750 - val_mse: 652193.3750
Epoch 54/100
235/235 [==============================] - 0s 2ms/step - loss: 673878.9375 - mse: 673878.9375 - val_loss: 818071.0000 - val_mse: 818071.0000
Epoch 55/100
235/235 [==============================] - 0s 2ms/step - loss: 712443.6875 - mse: 712443.6875 - val_loss: 699486.9375 - val_mse: 699486.9375
Epoch 56/100
235/235 [==============================] - 0s 2ms/step - loss: 686081.1875 - mse: 686081.1875 - val_loss: 720592.8750 - val_mse: 720592.8750
Epoch 57/100
235/235 [==============================] - 0s 2ms/step - loss: 799082.3750 - mse: 799082.3750 - val_loss: 849058.0625 - val_mse: 849058.0625
Epoch 58/100
235/235 [==============================] - 0s 2ms/step - loss: 785391.0625 - mse: 785391.0625 - val_loss: 1233627.1250 - val_mse: 1233627.1250
Epoch 59/100
235/235 [==============================] - 0s 2ms/step - loss: 699573.5000

In [41]:
loss, mse = model.evaluate(x_test_scaled, y_test, batch_size=1)
print('loss : ', loss)
print('mse : ', mse)

127/127 [==============================] - 0s 915us/step - loss: 1472975.3750 - mse: 1472975.3750
loss :  1472975.375
mse :  1472975.375


In [43]:
y_pred = model.predict(x_test_scaled)

In [44]:
for i in range(5):
    print('종가 : ', y_test[i], '/ 예측가 : ', y_pred[i])

종가 :  [52200.] / 예측가 :  [52228.31]
종가 :  [41450.] / 예측가 :  [39756.027]
종가 :  [49650.] / 예측가 :  [49988.043]
종가 :  [44800.] / 예측가 :  [45449.777]
종가 :  [49500.] / 예측가 :  [48378.645]
